TODO:

- [ ] Update Planning TODO list
- [ ] Create script version

# Executive Summary

This notebook will attempt to answer the following research question:

    What's money got to do with it?

## PLANNING

- [X] Planning
    - [X] import libraries/packages
    - [X] configure notebook environment
    - [X] define helper functions
- [X] Acquire data
    - [X] get PEIMS financial data
    - [X] get STAAR performance data
    - [X] get ETHNICITY data
- [X] Prepare the data
    - [X] prepare ETHNICITY data
        - [X] remove the nans (-999)
        - [X] get rid of unwanted columns
    - [X] prepare STAAR data
        - [X] get rid of duplicates
        - [X] get rid of unwanted columns
        - [X] get rid of NaNs
        - [X] create new columns
        - [X] merge three datasets together
    - [X] prepare PEIMS data
        - [X] get rid of unwanted columns
        - [x] get rid of NaNs
         
*First, let's prepare the notebook environment*

In [1]:
# for manipulating dataframes
import pandas as pd
import numpy as np

# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## ACQUIRE DATA

The datasets can be downloaded here:

- https://tea.texas.gov/finance-and-grants/state-funding/state-funding-reports-and-data/peims-financial-data-downloads  
- https://www.kaggle.com/datasets/9e3ce42f60ded3ba2a6dd890993493f2c4b284c5cfa035d711bd98fa3359924c?resource=download  
- https://rptsvr1.tea.texas.gov/adhocrpt/adste.html

In [2]:
# Get the ETHNICITY, PEIMS and STAAR datasets

peims_df = pd.read_csv('../data/in/financial/2007-2022-summarized-peims-financial-data.csv')
staar_df1 = pd.read_csv('../data/in/performance/tidy_campstaar1_2012to2019.csv')
staar_df2 = pd.read_csv('../data/in/performance/tidy_campstaar2_2013to2019.csv')
ethnic_df = pd.read_csv('../data/in/ethnicity/Enrollment Report_Statewide_Districts_Grade_Ethnicity_2018-2019.csv', skiprows=4)

C:\Users\Dd\AppData\Local\Temp\ipykernel_14732\1375482610.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  staar_df1 = pd.read_csv('../data/in/performance/tidy_campstaar1_2012to2019.csv')


## PREPARATION

### ETHNICITY Dataset

In [3]:
ethnic_df.head()

,YEAR,REGION,COUNTY NAME,DISTRICT,DISTRICT NAME,ETHNICITY,CHARTER STATUS,GRADE,ENROLLMENT
0,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Asian,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
1,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Black or African American,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
2,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Two or more races,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
3,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,White,TRADITIONAL ISD/CSD,Pre-kindergarten,-999.0
4,2018-2019,7.0,ANDERSON COUNTY,1902.0,CAYUGA ISD,Black or African American,TRADITIONAL ISD/CSD,Kindergarten,2.0


In [4]:
ethnic_df = ethnic_df[['DISTRICT',
                       'ETHNICITY',
                       'ENROLLMENT',
                      ]]

In [5]:
ethnic_df.head()

,DISTRICT,ETHNICITY,ENROLLMENT
0,1902.0,Asian,-999.0
1,1902.0,Black or African American,-999.0
2,1902.0,Two or more races,-999.0
3,1902.0,White,-999.0
4,1902.0,Black or African American,2.0


In [6]:
ethnic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66510 entries, 0 to 66509
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DISTRICT    66508 non-null  float64
 1   ETHNICITY   66508 non-null  object 
 2   ENROLLMENT  66508 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.5+ MB


In [7]:
# get rid of invalid enrollment values where no data is provided (-999)
ethnic_df = ethnic_df[ethnic_df['ENROLLMENT'] != -999]

In [8]:
ethnic_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60005 entries, 4 to 66509
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DISTRICT    60003 non-null  float64
 1   ETHNICITY   60003 non-null  object 
 2   ENROLLMENT  60003 non-null  float64
dtypes: float64(2), object(1)
memory usage: 1.8+ MB


In [9]:
# create a column with number of white students enrolled
ethnic_df['ENROLLED_WHITE'] = np.where(ethnic_df['ETHNICITY'] == 'White', ethnic_df['ENROLLMENT'], 0)

# create a column with number of non-white students enrolled
ethnic_df['ENROLLED_X'] = np.where(ethnic_df['ETHNICITY'] != 'White', ethnic_df['ENROLLMENT'], 0)

In [10]:
ethnic_df.head(99)

,DISTRICT,ETHNICITY,ENROLLMENT,ENROLLED_WHITE,ENROLLED_X
4,1902.0,Black or African American,2.0,0.0,2.0
5,1902.0,Hispanic/Latino,6.0,0.0,6.0
6,1902.0,White,30.0,30.0,0.0
9,1902.0,Black or African American,2.0,0.0,2.0
10,1902.0,Hispanic/Latino,3.0,0.0,3.0
11,1902.0,White,41.0,41.0,0.0
12,1902.0,Black or African American,3.0,0.0,3.0
13,1902.0,Hispanic/Latino,3.0,0.0,3.0
14,1902.0,Two or more races,2.0,0.0,2.0
15,1902.0,White,37.0,37.0,0.0


In [11]:
ethnic_df = ethnic_df.drop(columns=['ENROLLMENT', 'ETHNICITY'])

In [12]:
ethnic_df.head()

,DISTRICT,ENROLLED_WHITE,ENROLLED_X
4,1902.0,0.0,2.0
5,1902.0,0.0,6.0
6,1902.0,30.0,0.0
9,1902.0,0.0,2.0
10,1902.0,0.0,3.0


In [13]:
# turn dataset into something-level data
ethnic_df = ethnic_df.groupby(['DISTRICT',
                              ])[["ENROLLED_WHITE",
                                  "ENROLLED_X"]].sum().reset_index()

In [14]:
ethnic_df.head()

,DISTRICT,ENROLLED_WHITE,ENROLLED_X
0,1902.0,447.0,76.0
1,1903.0,981.0,263.0
2,1904.0,598.0,162.0
3,1906.0,245.0,90.0
4,1907.0,907.0,2309.0


In [15]:
# create column with percentage of non-white students and drop the numerator and denominator
ethnic_df['XFACTOR'] = ethnic_df["ENROLLED_X"] / (ethnic_df["ENROLLED_WHITE"] + ethnic_df["ENROLLED_X"] )
ethnic_df = ethnic_df.drop(columns=['ENROLLED_WHITE', 'ENROLLED_X'])

In [16]:
ethnic_df[ethnic_df['DISTRICT'] == 1902.0]

,DISTRICT,XFACTOR
0,1902.0,0.145315


In [17]:
# for readability
ethnic_df = ethnic_df.rename(columns={'DISTRICT':'District',
                                      'XFACTOR':'Percentage of Non-White Students'
                        })

In [18]:
ethnic_df['District'] = ethnic_df['District'].astype('int64')

In [19]:
ethnic_df.dtypes

District                              int64
Percentage of Non-White Students    float64
dtype: object

In [20]:
# prepend the district numbers with 0's
ethnic_df['District'] = ethnic_df['District'].apply(lambda x: '{0:0>6}'.format(x))

In [21]:
ethnic_df.head()

,District,Percentage of Non-White Students
0,001902,0.145315
1,001903,0.211415
2,001904,0.213158
3,001906,0.268657
4,001907,0.717973


### STAAR Dataset

In [22]:
# consolidate the two files into one dataframe
staar_df = pd.concat([staar_df1, staar_df2])

In [23]:
# get rid of duplciates
staar_df = staar_df.drop_duplicates(keep='first')

In [24]:
columns_to_drop = ['data_release',
                   'data_category',
                   'data_level',
                   'release_year'
                  ]

# get rid of unnecessary columns
staar_df.drop(columns=columns_to_drop, inplace=True)

In [25]:
# prepend campus number with 0's
staar_df['campus_number'] = staar_df['campus_number'].apply(lambda x: '{0:0>9}'.format(x))

In [26]:
# drop 'new_rate'
staar_df.drop(columns=['new_rate'], inplace=True)

In [27]:
# get the district number
staar_df['district'] = staar_df['campus_number'].str[:6]

In [28]:
# turn dataset into district-level data
staar = staar_df.groupby(['test_year',
                          'district',
                          'grade_level',
                          'subject',
                          'proficiency',
                          'demog'
                         ])[["numerator", "denominator"]].sum().reset_index()

In [29]:
# remove subsets
staar = staar[staar['grade_level'] == 'all']
staar = staar[staar['subject'] == 'all_subjects']
staar = staar[staar['demog'] == 'all_students']
staar = staar.drop(columns=['grade_level',
                            'subject',
                            'demog'
                           ])

In [30]:
staar.head()

,test_year,district,proficiency,numerator,denominator
99,2012,001902,approaches,1030,1219
110,2012,001902,masters,144,1219
120,2012,001902,meets,471,1219
372,2012,001903,approaches,1796,2279
383,2012,001903,masters,203,2279


In [31]:
# create new column
staar['total_student'] = staar['denominator']

In [32]:
staar['proficiency'].value_counts()

proficiency
approaches    9641
masters       7126
meets         4780
Name: count, dtype: int64

In [33]:
# create column with number of not passing students
staar['not_passing'] = np.where(staar['proficiency'] == 'approaches', staar['numerator'], 0)

# create column with number of passing students
staar['passing'] = np.where(staar['proficiency'] != 'approaches', staar['numerator'], 0)

In [34]:
# isolate the 2019 test year and drop the year column
staar = staar[staar['test_year'] == 2019]

In [35]:
staar = staar.drop(columns=['test_year',
                            'proficiency',
                            'numerator',
                            'denominator',
                            'not_passing'
                           ])

In [36]:
staar[staar['district'] == '001902']

,district,total_student,passing
2682938,001902,932,0
2682951,001902,932,269
2682964,001902,932,573


In [37]:
# for readability
staar = staar.rename(columns={'district':'District',
                        'total_student':'Total Number of Students',
                        'passing':'Total Number of Passing Students'
                        })

In [38]:
# Turn dataset into something-level data
staar = staar.groupby(['District',
                       'Total Number of Students',
                      ])[["Total Number of Passing Students"]].sum().reset_index()

In [39]:
staar.head(10)

,District,Total Number of Students,Total Number of Passing Students
0,001902,932,842
1,001903,1991,1651
2,001904,1365,1064
3,001906,569,420
4,001907,5497,3274
5,001908,2467,1240
6,001909,635,489
7,002901,7268,4476
8,003801,1774,1493
9,003902,4693,1461


### PEIMS Dataset

In [40]:
peims_df.head()

,DISTRICT NUMBER,DISTRICT NAME,YEAR,GEN FUNDS-LOCAL TAX REVENUE FROM M&O,ALL FUNDS-LOCAL TAX REVENUE FROM M&O,GEN FUNDS-STATE REVENUE,ALL FUNDS-STATE REVENUE,GEN FUNDS-FEDERAL REVENUE,ALL FUNDS-FEDERAL REVENUE,GEN FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,ALL FUNDS-OTHER LOCAL & INTERMEDIATE REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE,GEN FUNDS-LOCAL PROPERTY TAXES FROM I&S,ALL FUNDS-LOCAL PROPERTY TAXES FROM I&S,GEN FUNDS-STATE DEBT FUNDS,ALL FUNDS-STATE DEBT FUNDS,GEN FUNDS-OTHER RECEIPTS,ALL FUNDS-OTHER RECEIPTS,GEN FUNDS-OTHER REVENUE,ALL FUNDS-OTHER REVENUE,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE,GEN FUNDS-EQUITY TRANSFERS,ALL FUNDS-EQUITY TRANSFERS,GEN FUNDS-TOT DEBT SERV FIN AND TRS EST REV,ALL FUNDS-TOT DEBT SERV FIN AND TRS EST REV,GEN FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECAPTUR,ALL FUNDS-TOTAL OPERATING REVENUE AND OTHER REVENUE AND RECAPTUR,GEN FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,ALL FUNDS-DEBT SERVICE FINANCING RELATED REVENUE,GEN FUNDS-ESTIMATED STATE TRS CONTRIBUTIONS,ALL FUNDS-ESTIMATED STATE TRS CONTRIBUTIONS,GEN FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,ALL FUNDS-TOTAL DEBT SERVICE FINANCING AND TRS ESTIMATE REVENUE,"GEN FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN","ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN",GEN FUNDS-TOTAL PAYROLL EXPENDITURES,ALL FUNDS-TOTAL PAYROLL EXPENDITURES,GEN FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,ALL FUNDS-TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,GEN FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,ALL FUNDS-TOTAL SUPPLIES & MATERIALS EXPENDITURES,GEN FUNDS-TOTAL OTHER OPERATING EXPENDITURES,ALL FUNDS-TOTAL OTHER OPERATING EXPENDITURES,GEN FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,ALL FUNDS-TOTAL OPERATING EXPENDITURES BY OBJ,GEN FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,ALL FUNDS-TOTAL DEBT SERVICE EXPEND BY OBJ,GEN FUNDS-TOTAL CAPITAL PROJECTS EXPEND BY OBJ,ALL FUNDS-TOTAL CAPITAL PROJECTS EXPEND BY OBJ,GEN FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER EXPENDITURES BY OBJ,GEN FUNDS-TOTAL NON-OPER AND OPER EXPENDITURES BY OBJ,ALL FUNDS-TOTAL NON-OPER AND OPER EXPENDITURES BY OBJ,"GEN FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","ALL FUNDS-INSTRUCTION + TRANSFER EXPEND-FCT11,95","GEN FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","ALL FUNDS-INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","GEN FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","ALL FUNDS-CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","GEN FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","ALL FUNDS-INSTRUC LEADERSHIP EXPEND, FCT21","GEN FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","ALL FUNDS-CAMPUS ADMINISTRATION EXPEND, FCT23","GEN FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","ALL FUNDS-GUIDANCE & COUNSELING SERVICES EXP, FCT31","GEN FUNDS-SOCIAL WORK SERVICES EXP, FCT32","ALL FUNDS-SOCIAL WORK SERVICES EXP, FCT32","GEN FUNDS-HEALTH SERVICES EXP, FCT33","ALL FUNDS-HEALTH SERVICES EXP, FCT33","GEN FUNDS-TRANSPORTATION EXPENDITURES, FCT34","ALL FUNDS-TRANSPORTATION EXPENDITURES, FCT34","GEN FUNDS-FOOD SERVICE EXPENDITURES, FCT35","ALL FUNDS-FOOD SERVICE EXPENDITURES, FCT35","GEN FUNDS-EXTRACURRICULAR EXPENDITURES, FCT36","ALL FUNDS-EXTRACURRICULAR EXPENDITURES,FCT36","GEN FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,92","ALL FUNDS-GENERAL ADMINISTRAT EXPEND-FCT41,92","GEN FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","ALL FUNDS-PLANT MAINTENANCE/OPERA EXPEND, FCT51","GEN FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","ALL FUNDS-SECURITY/MONITORING SERVICE EXPEND, FCT52","GEN FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","ALL FUNDS-DATA PROCESSING SERVICES EXPEND, FCT53","GEN FUNDS-COMMUNITY SERVICES, FCT61","ALL FUNDS-COMMUNITY SERVICES, FCT61",GEN FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,ALL FUNDS-TOTAL OPERATE EXPEND BY FUNCTION,GEN FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65XX),ALL FUNDS-NON-OPER EXP BY FUNCTION(1X-9X)(65X

In [41]:
peims = peims_df.drop(columns=['DISTRICT NAME'])

In [42]:
# Remove all columns between column name 'A' to 'B'
peims = peims.drop(peims.loc[:, 'GEN FUNDS-LOCAL TAX REVENUE FROM M&O':'ALL FUNDS-TOTAL OPERATING, OTR, DEBT SERV FIN, AND TRS EST REVEN'].columns, axis=1)

In [43]:
# isolate 2019 test year
peims = peims[peims['YEAR'] == 2019]
peims = peims.drop(columns=['YEAR'])

In [44]:
# remove columns that starts with 'GEN'
columns_to_keep = [c for c in peims.columns if c.lower()[:3] != 'gen']
peims = peims[columns_to_keep]

In [45]:
# remove prefix 'all funds'
peims.columns = peims.columns.str.replace("ALL FUNDS-", "")

In [46]:
peims['DISTRICT NUMBER'] = peims['DISTRICT NUMBER'].astype('str')

In [47]:
# padd district numbers with 0's
peims['DISTRICT NUMBER'] = peims['DISTRICT NUMBER'].str.zfill(6)

In [48]:
peims.columns

Index(['DISTRICT NUMBER', 'TOTAL PAYROLL EXPENDITURES',
       'TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES',
       'TOTAL SUPPLIES & MATERIALS EXPENDITURES',
       'TOTAL OTHER OPERATING EXPENDITURES',
       'TOTAL OPERATING EXPENDITURES BY OBJ',
       'TOTAL DEBT SERVICE EXPEND BY OBJ',
       'TOTAL CAPITAL PROJECTS EXPEND BY OBJ',
       'TOTAL NON-OPER EXPENDITURES BY OBJ',
       'TOTAL NON-OPER AND OPER EXPENDITURES BY OBJ',
       'INSTRUCTION + TRANSFER EXPEND-FCT11,95',
       'INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12',
       'CURRICULUM/STAFF DEVELOPMENT EXP, FCT13',
       'INSTRUC LEADERSHIP EXPEND, FCT21',
       'CAMPUS ADMINISTRATION EXPEND, FCT23',
       'GUIDANCE & COUNSELING SERVICES EXP, FCT31',
       'SOCIAL WORK SERVICES EXP, FCT32', 'HEALTH SERVICES EXP, FCT33',
       'TRANSPORTATION EXPENDITURES, FCT34',
       'FOOD SERVICE EXPENDITURES, FCT35',
       'EXTRACURRICULAR EXPENDITURES,FCT36',
       'GENERAL ADMINISTRAT EXPEND-FCT41,92',
       

In [49]:
peims = peims.drop(peims.loc[:, 'TOTAL OPERATING EXPENDITURES BY OBJ':'TOTAL NON-OPER AND OPER EXPENDITURES BY OBJ'].columns, axis=1)

In [50]:
peims = peims.drop(peims.loc[:, 'TOTAL OPERATE EXPEND BY FUNCTION':'TOT OPER AND NON-OPER EXP BY FUNCTION'].columns, axis=1)

In [51]:
peims = peims.drop(peims.loc[:, 'NON OPER EXP BY PIC(65XX)':'EQUITY TRANSFERS FOR TD'].columns, axis=1)

In [52]:
peims = peims.drop(peims.loc[:, 'INTERGOVERN CHARGES EXPEND':'FALL SURVEY ENROLLMENT'].columns, axis=1)

In [53]:
peims.columns

Index(['DISTRICT NUMBER', 'TOTAL PAYROLL EXPENDITURES',
       'TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES',
       'TOTAL SUPPLIES & MATERIALS EXPENDITURES',
       'TOTAL OTHER OPERATING EXPENDITURES',
       'INSTRUCTION + TRANSFER EXPEND-FCT11,95',
       'INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12',
       'CURRICULUM/STAFF DEVELOPMENT EXP, FCT13',
       'INSTRUC LEADERSHIP EXPEND, FCT21',
       'CAMPUS ADMINISTRATION EXPEND, FCT23',
       'GUIDANCE & COUNSELING SERVICES EXP, FCT31',
       'SOCIAL WORK SERVICES EXP, FCT32', 'HEALTH SERVICES EXP, FCT33',
       'TRANSPORTATION EXPENDITURES, FCT34',
       'FOOD SERVICE EXPENDITURES, FCT35',
       'EXTRACURRICULAR EXPENDITURES,FCT36',
       'GENERAL ADMINISTRAT EXPEND-FCT41,92',
       'PLANT MAINTENANCE/OPERA EXPEND, FCT51',
       'SECURITY/MONITORING SERVICE EXPEND, FCT52',
       'DATA PROCESSING SERVICES EXPEND, FCT53', 'COMMUNITY SERVICES, FCT61',
       'REGULAR PROGRAM EXPEND--11', 'GIFTED/TALENTED PROGRAM EX

In [54]:
# peims = peims.drop(peims.loc[:, 'PREKINDERGARTEN EXPEND BILINGUAL--32':'PREKINDERGARTEN EXPEND SPECIAL ED--32'].columns, axis=1)

In [55]:
peims = peims.rename(columns={'DISTRICT NUMBER':'DISTRICT'})

In [56]:
peims.head()

,DISTRICT,TOTAL PAYROLL EXPENDITURES,TOTAL PROFESSIONAL & CONTRACTED SERVICES EXPENDITURES,TOTAL SUPPLIES & MATERIALS EXPENDITURES,TOTAL OTHER OPERATING EXPENDITURES,"INSTRUCTION + TRANSFER EXPEND-FCT11,95","INSTRUC RESOURCE MEDIA SERVICE EXP, FCT12","CURRICULUM/STAFF DEVELOPMENT EXP, FCT13","INSTRUC LEADERSHIP EXPEND, FCT21","CAMPUS ADMINISTRATION EXPEND, FCT23","GUIDANCE & COUNSELING SERVICES EXP, FCT31","SOCIAL WORK SERVICES EXP, FCT32","HEALTH SERVICES EXP, FCT33","TRANSPORTATION EXPENDITURES, FCT34","FOOD SERVICE EXPENDITURES, FCT35","EXTRACURRICULAR EXPENDITURES,FCT36","GENERAL ADMINISTRAT EXPEND-FCT41,92","PLANT MAINTENANCE/OPERA EXPEND, FCT51","SECURITY/MONITORING SERVICE EXPEND, FCT52","DATA PROCESSING SERVICES EXPEND, FCT53","COMMUNITY SERVICES, FCT61",REGULAR PROGRAM EXPEND--11,GIFTED/TALENTED PROGRAM EXPEND--21,CAREER & TECHNOLOGY PGM EXPEND--22,STUDENTS WITH DISABILITIES PGM EXPEND--23,"STATE COMPENSATORY ED EXPEND--24, 26, 28, 29, 30, 34","BILINGUAL PROGRAM EXP--25, 35",HIGH SCHOOL ALLOTMENT PROGRAM--31,"PREKINDERGARTEN--32,35",ATHLETICS PROGRAM EXPEND--91,UNDISTRIBUTED PROGRAM EXP--99,TOTAL PROGRAM OPERATING EXPENDITURES,TOTAL OTHER USES
12,001902,"$6,025,217","$1,075,904","$648,206","$809,559","$4,649,118","$66,490","$4,986","$270,353","$306,385","$998,314",$0,"$37,882","$293,070","$287,406","$413,755","$284,553","$773,085",$0,"$173,489",$0,"$2,778,638","$3,968","$251,350","$3,005,575","$273,747","$9,599","$40,285","$32,890","$304,174","$1,858,660","$8,558,886","$48,633"
28,001903,"$9,093,950","$1,514,689","$784,631","$303,052","$7,043,892","$117,860","$33,175","$66,374","$574,699","$202,086",$0,"$33,657","$422,887","$630,202","$598,484","$558,948","$1,248,908","$13,530","$151,120",$500,"$5,313,722",$93,"$852,319","$1,028,587","$799,037",$0,"$101,243",$0,"$339,045","$3,262,276","$11,696,322","$102,465"
44,001904,"$6,659,596","$927,209","$937,810","$278,109","$4,611,747","$51,126","$157,830",$0,"$466,345","$199,338",$0,"$102,385","$38,800","$411,195","$754,465","$539,512","$1,014,501","$45,482","$409,998",$0,"$3,945,494","$10,154","$552,217","$726,827","$377,013",$0,"$59,567","$114,404","$571,388","$2,445,660","$8,802,724",$481
60,001906,"$3,134,475","$373,513","$408,024","$105,878","$2,087,166","$19,990",$0,"$7,905","$379,101","$75,235",$0,"$40,628","$148,301","$257,465","$210,240","$201,520","$465,549","$10,415","$118,375",$0,"$1,499,301","$14,498","$164,641","$447,072","$402,415","$2,706","$48,748","$29,920",$0,"$1,412,589","$4,021,890","$53,786"
76,001907,"$25,587,063","$5,603,896","$4,134,969","$1,048,416","$18,807,861","$167,823","$535,649","$1,033,275","$2,201,907","$1,443,630","$170,074","$208,736","$1,442,619","$2,071,781","$1,422,648","$1,287,489","$3,937,087","$242,658","$1,006,175","$394,932","$15,527,277","$39,671","$1,625,090","$2,422,707","$3,147,717","$231,026","$302,531","$923,035","$1,214,433","$10,940,857","$36,374,344",$0


In [57]:
# Remove word 'EXPENDITURES'
peims.columns = peims.columns.str.replace("EXPENDITURES", "")

In [58]:
# Remove word 'EXPEND'
peims.columns = peims.columns.str.replace("EXPEND", "")

In [59]:
# Remove word 'EXP'
peims.columns = peims.columns.str.replace("EXP", "")

In [60]:
# Remove word 'TOTAL'
peims.columns = peims.columns.str.replace("TOTAL", "")

In [61]:
# Remove word 'FCT'
peims.columns = peims.columns.str.replace("FCT", "")

In [62]:
# Remove dougble dashes
peims.columns = peims.columns.str.replace("--", "-")

In [63]:
# Remove double space opposite comma
peims.columns = peims.columns.str.replace(" , ", ",")

In [64]:
# Remove leading and treiling spaces
peims.columns = peims.columns.str.strip()

In [65]:
peims = peims.rename(columns={'DISTRICT':'District',
                              'PAYROLL':'Payroll',
                              'PROFESSIONAL & CONTRACTED SERVICES':'Professional and Contracted',
                              'SUPPLIES & MATERIALS':'Supplies and Materials',
                              'OTHER OPERATING':'Other Operating',
                              'INSTRUCTION + TRANSFER -11,95':'Instruction and Transfer',
                              'INSTRUC RESOURCE MEDIA SERVICE, 12':'Instructional Resource Media',
                              'CURRICULUM/STAFF DEVELOPMENT,13':'Curriculum/Staff Development',
                              'INSTRUC LEADERSHIP,21':'Instructional Leadership',
                              'CAMPUS ADMINISTRATION,23':'Campus Administration',
                              'GUIDANCE 7 COUNSELING SERVICES,31':'Guidance & Counseling',
                              'SOCIAL WORK SERVICES,32':'Social Work',
                              'HEALTH SERVICES,33':'Health Services',
                              'TRANSPORTATION,34':'Transportation',
                              'FOOD SERVICE,35':'Food Service',
                              'EXTRACURRICULAR ,36':'Extracurricular',
                              'GENERAL ADMINISTRAT -41,80,92':'General Administration',
                              'PLANT MAINTENANCE/OPERA,51':'Plant Maintenance/Operation',
                              'SECURITY/MONITORING SERVICE,5':'Security and Monitoring',
                              'DATA PROCESSING SERVICES, 53':'Data Processing',
                              'COMMUNITY SERVICES, 61':'Community Services',
                              'REGULAR PROGRAM -11':'Regular Program',
                              'GIFTED/TALENTED PROGRAM -21':'Gifted and Talented Program',
                              'CAREER & TECHNOLOGY PGM -22':'Career and Technology Program',
                              'STUDENTS WITH DISABILITIES PGM -23':'Students with Disabilities',
                              'STATE COMPENSATORY ED -24, 29, 30, 34':'State Compensatory Education',
                              'BILINGUAL PROGRAM -25':'Bilingual Program',
                              'HIGH SCHOOL ALLOTMENT PROGRAM-91':'High School Allotment',
                              'PREKINDERGARTEN-32,35':'Pre-K',
                              'ATHLETICS PROGRAM-91':'Athletics Program',
                              'UNDISTRIBUTED PROGRAM -99':'Undistributed Program',
                              'PROGRAM OPERATING':'Total Program Expenditures',
                              'OTHER USES':'Other Uses'
                             })

In [66]:
peims.head().T

,12,28,44,60,76
District,001902,001903,001904,001906,001907
Payroll,"$6,025,217","$9,093,950","$6,659,596","$3,134,475","$25,587,063"
Professional and Contracted,"$1,075,904","$1,514,689","$927,209","$373,513","$5,603,896"
Supplies and Materials,"$648,206","$784,631","$937,810","$408,024","$4,134,969"
Other Operating,"$809,559","$303,052","$278,109","$105,878","$1,048,416"
Instruction and Transfer,"$4,649,118","$7,043,892","$4,611,747","$2,087,166","$18,807,861"
"INSTRUC RESOURCE MEDIA SERVICE,12","$66,490","$117,860","$51,126","$19,990","$167,823"
Curriculum/Staff Development,"$4,986","$33,175","$157,830",$0,"$535,649"
Instructional Leadership,"$270,353","$66,374",$0,"$7,905","$1,033,275"
Campus Administration,"$306,385","$574,699","$466,345","$379,101","$2,201,907"


In [67]:
peims.columns

Index(['District', 'Payroll', 'Professional and Contracted',
       'Supplies and Materials', 'Other Operating', 'Instruction and Transfer',
       'INSTRUC RESOURCE MEDIA SERVICE,12', 'Curriculum/Staff Development',
       'Instructional Leadership', 'Campus Administration',
       'GUIDANCE & COUNSELING SERVICES,31', 'Social Work', 'Health Services',
       'Transportation', 'Food Service', 'Extracurricular',
       'GENERAL ADMINISTRAT -41,92', 'Plant Maintenance/Operation',
       'SECURITY/MONITORING SERVICE,52', 'DATA PROCESSING SERVICES,53',
       'Community Services', 'Regular Program', 'Gifted and Talented Program',
       'Career and Technology Program', 'Students with Disabilities',
       'STATE COMPENSATORY ED -24, 26, 28, 29, 30, 34',
       'BILINGUAL PROGRAM -25, 35', 'HIGH SCHOOL ALLOTMENT PROGRAM-31',
       'Pre-K', 'ATHLETICS PROGRAM -91', 'Undistributed Program',
       'Total Program Expenditures', 'Other Uses'],
      dtype='object')

In [68]:
peims = peims.rename(columns={'INSTRUC RESOURCE MEDIA SERVICE,12':'Instructional Resource Media Service',
                              'GUIDANCE & COUNSELING SERVICES,31':'Guidance and Counseling',
                              'GENERAL ADMINISTRAT -41,92':'General Adminstration',
                              'SECURITY/MONITORING SERVICE,52':'Security and Monitoring Service',
                              'DATA PROCESSING SERVICES,53':'Data Processing Services',
                              'STATE COMPENSATORY ED -24, 26, 28, 29, 30, 34':'State Compensatory Education',
                              'BILINGUAL PROGRAM -25, 35':'Bilingual Program',
                              'HIGH SCHOOL ALLOTMENT PROGRAM-31':'High School Allotment Program',
                              'ATHLETICS PROGRAM -91':'Atheletics Program'
                             })

In [69]:
peims.head().T

,12,28,44,60,76
District,001902,001903,001904,001906,001907
Payroll,"$6,025,217","$9,093,950","$6,659,596","$3,134,475","$25,587,063"
Professional and Contracted,"$1,075,904","$1,514,689","$927,209","$373,513","$5,603,896"
Supplies and Materials,"$648,206","$784,631","$937,810","$408,024","$4,134,969"
Other Operating,"$809,559","$303,052","$278,109","$105,878","$1,048,416"
Instruction and Transfer,"$4,649,118","$7,043,892","$4,611,747","$2,087,166","$18,807,861"
Instructional Resource Media Service,"$66,490","$117,860","$51,126","$19,990","$167,823"
Curriculum/Staff Development,"$4,986","$33,175","$157,830",$0,"$535,649"
Instructional Leadership,"$270,353","$66,374",$0,"$7,905","$1,033,275"
Campus Administration,"$306,385","$574,699","$466,345","$379,101","$2,201,907"


In [70]:
peims.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 12 to 19415
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   District                              1200 non-null   object
 1   Payroll                               1200 non-null   object
 2   Professional and Contracted           1200 non-null   object
 3   Supplies and Materials                1200 non-null   object
 4   Other Operating                       1200 non-null   object
 5   Instruction and Transfer              1200 non-null   object
 6   Instructional Resource Media Service  1200 non-null   object
 7   Curriculum/Staff Development          1200 non-null   object
 8   Instructional Leadership              1200 non-null   object
 9   Campus Administration                 1200 non-null   object
 10  Guidance and Counseling               1200 non-null   object
 11  Social Work                      

In [71]:
def clean_currency(string_num):
    """ If the value is a string, remove currency symbol
    and commas, otherwise, do nothing
    """
    if isinstance(string_num, str):
        return(string_num.replace('$', '').replace(',', ''))
    return(string_num)

In [72]:
currency_columns = list(peims.columns)
currency_columns

['District',
 'Payroll',
 'Professional and Contracted',
 'Supplies and Materials',
 'Other Operating',
 'Instruction and Transfer',
 'Instructional Resource Media Service',
 'Curriculum/Staff Development',
 'Instructional Leadership',
 'Campus Administration',
 'Guidance and Counseling',
 'Social Work',
 'Health Services',
 'Transportation',
 'Food Service',
 'Extracurricular',
 'General Adminstration',
 'Plant Maintenance/Operation',
 'Security and Monitoring Service',
 'Data Processing Services',
 'Community Services',
 'Regular Program',
 'Gifted and Talented Program',
 'Career and Technology Program',
 'Students with Disabilities',
 'State Compensatory Education',
 'Bilingual Program',
 'High School Allotment Program',
 'Pre-K',
 'Atheletics Program',
 'Undistributed Program',
 'Total Program Expenditures',
 'Other Uses']

In [73]:
currency_columns = currency_columns[1:]
currency_columns

['Payroll',
 'Professional and Contracted',
 'Supplies and Materials',
 'Other Operating',
 'Instruction and Transfer',
 'Instructional Resource Media Service',
 'Curriculum/Staff Development',
 'Instructional Leadership',
 'Campus Administration',
 'Guidance and Counseling',
 'Social Work',
 'Health Services',
 'Transportation',
 'Food Service',
 'Extracurricular',
 'General Adminstration',
 'Plant Maintenance/Operation',
 'Security and Monitoring Service',
 'Data Processing Services',
 'Community Services',
 'Regular Program',
 'Gifted and Talented Program',
 'Career and Technology Program',
 'Students with Disabilities',
 'State Compensatory Education',
 'Bilingual Program',
 'High School Allotment Program',
 'Pre-K',
 'Atheletics Program',
 'Undistributed Program',
 'Total Program Expenditures',
 'Other Uses']

In [74]:
# convert the currencies to int
for col in currency_columns:
    peims[col] = peims[col].apply(clean_currency).astype('int')

In [75]:
peims.info()
peims.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 12 to 19415
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   District                              1200 non-null   object
 1   Payroll                               1200 non-null   int32 
 2   Professional and Contracted           1200 non-null   int32 
 3   Supplies and Materials                1200 non-null   int32 
 4   Other Operating                       1200 non-null   int32 
 5   Instruction and Transfer              1200 non-null   int32 
 6   Instructional Resource Media Service  1200 non-null   int32 
 7   Curriculum/Staff Development          1200 non-null   int32 
 8   Instructional Leadership              1200 non-null   int32 
 9   Campus Administration                 1200 non-null   int32 
 10  Guidance and Counseling               1200 non-null   int32 
 11  Social Work                      

,District,Payroll,Professional and Contracted,Supplies and Materials,Other Operating,Instruction and Transfer,Instructional Resource Media Service,Curriculum/Staff Development,Instructional Leadership,Campus Administration,Guidance and Counseling,Social Work,Health Services,Transportation,Food Service,Extracurricular,General Adminstration,Plant Maintenance/Operation,Security and Monitoring Service,Data Processing Services,Community Services,Regular Program,Gifted and Talented Program,Career and Technology Program,Students with Disabilities,State Compensatory Education,Bilingual Program,High School Allotment Program,Pre-K,Atheletics Program,Undistributed Program,Total Program Expenditures,Other Uses
12,001902,6025217,1075904,648206,809559,4649118,66490,4986,270353,306385,998314,0,37882,293070,287406,413755,284553,773085,0,173489,0,2778638,3968,251350,3005575,273747,9599,40285,32890,304174,1858660,8558886,48633
28,001903,9093950,1514689,784631,303052,7043892,117860,33175,66374,574699,202086,0,33657,422887,630202,598484,558948,1248908,13530,151120,500,5313722,93,852319,1028587,799037,0,101243,0,339045,3262276,11696322,102465
44,001904,6659596,927209,937810,278109,4611747,51126,157830,0,466345,199338,0,102385,38800,411195,754465,539512,1014501,45482,409998,0,3945494,10154,552217,726827,377013,0,59567,114404,571388,2445660,8802724,481
60,001906,3134475,373513,408024,105878,2087166,19990,0,7905,379101,75235,0,40628,148301,257465,210240,201520,465549,10415,118375,0,1499301,14498,164641,447072,402415,2706,48748,29920,0,1412589,4021890,53786
76,001907,25587063,5603896,4134969,1048416,18807861,167823,535649,1033275,2201907,1443630,170074,208736,1442619,2071781,1422648,1287489,3937087,242658,1006175,394932,15527277,39671,1625090,2422707,3147717,231026,302531,923035,1214433,10940857,36374344,0


In [76]:
ethnic_df.shape
staar.shape
peims.shape

(1201, 2)

(1251, 3)

(1200, 33)

In [77]:
# save intermediary files
ethnic_df.to_csv('../data/inter/clean_ethnic_2019.csv', index=False)
staar.to_csv('../data/inter/clean_staar_2019.csv', index=False)
peims.to_csv('../data/inter/clean_peims_2019.csv', index=False)